In [ ]:
# Initialization: Load shared functions and simulated data 

# Load shared functions
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

# Get simulated data from Github repository
if not os.path.exists("simulated-data-raw"):
    !git clone https://github.com/Fraud-Detection-Handbook/simulated-data-raw
        

In [ ]:
dir_input='./simulated-data-raw/data/' 

begin_date = "2023-12-01"
end_date = "2024-02-28"

print("Load  files")
%time transactions_df = read_from_files(dir_input, being_date, end_date)
print("{0} transactions loaded, containing {1} fraudulent transactions"\
      .format(len(transactions_df),transactions_df.tx_fraud.sum()))

In [ ]:
transactions_df.head()

In [ ]:
def is_weekend(tx_datetime):

    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekend = tx_datetime.weekday()
    # Binary value: 0 weekday, 1 if weekend
    is_weekend = weekday>=5

    return int(is_weekend)

In [ ]:
%time transactions_df['tx_during_weekend'] = transaction_df.tx_datetime\
                                            .apply(is_weekend)

In [ ]:
def is_night(tx_datetime):

    # Get the hour of the Transaction
    tx_hour = tx_datetime.hour
    # Binary Value: 1 if hour is less than 6, an 0 otherwise
    is_night = tx_hour<=6

    return int(is_night)

In [ ]:
%time transactions_df['tx_during_night'] = transactions_df.tx_datetime\
                                            .apply(is_night)

In [ ]:
transactions_df[transactions_df.tx_time_days>=30]

In [ ]:
def get_customer_spending_behavior_features(customer_transactions, windows_size_in_days=[1,7,30]):

    # Order Transactions Chronologically
    customer_transactions = customer_transactions.sort_values('tx_datetime')
    